In [3]:
# important modules for this HW
import bs4 # doc: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests as rq 
import re
import time
# previous useful modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [4]:
import copy
import datetime

# Loading the 2 dataset

In [ ]:
df1=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_1.csv')

In [ ]:
df2=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_2.csv')

In [ ]:
df=pd.concat([df1,df2],axis=0)

In [ ]:
df.head()

In [ ]:
a=list(df.cat.value_counts().index)

In [ ]:
a

In [ ]:
st=""
for i,item in enumerate(a):
    if i>1000 and i<1500:
        st+=item
        st+=" "
st

In [ ]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

# Links2run integration

### Loading links2run

In [ ]:
links2runs=pd.read_csv('../datasets/links2runs.csv')
del links2runs['Unnamed: 0']

In [ ]:
links2runs=links2runs.set_index(links2runs.URL)

In [ ]:
links2runs.head(3)

In [ ]:
links2runs.loc['http://services.datasport.com/2000/lauf/tdcn']

In [ ]:
links2runs.index.value_counts()

### Process link

In [ ]:
df.link=df.link.apply(lambda x: '/'.join(x.split('/')[:-1]))

In [ ]:
df.head(2)

### Groupby link

In [ ]:
links=list(df.link.value_counts().index)

In [ ]:
def add_date_name_to_group(x):
    link=x.link.iloc[0]
#     print(link)
#     print('\n\n\n\n')
    row_links2runs=links2runs.loc[link]
    try:
        row_links2runs.shape[1]
        if row_links2runs.shape[0]>1:
            row_links2runs=row_links2runs.iloc[0]
    except:
        pass
#     print(row_links2runs.shape)
    try:
        x['Date']=row_links2runs.Date
    except:
        print(row_links2runs)
        print(x)
        raise()
    x['Name']=row_links2runs.Name
    return x

In [ ]:
df.head(2)

In [ ]:
grouped_link=df.groupby(df.link)

In [ ]:
df=grouped_link.apply(add_date_name_to_group)

In [ ]:
del df['link']

In [ ]:
df.head(2)

# Delete duplicates
Some links of games are counted twice. In this case we check for duplicated rows and we remove them

In [ ]:
df=df.drop_duplicates(keep='first')

# Process temps-retards

In [ ]:
class Time:
    def __init__(self,time,mseconds=True):
        self.days,self.hours,self.minutes,self.seconds,self.mseconds=self.parse_time(time,split=True,mseconds=mseconds)
        
    def parse_time(self,time,check_only=False,split=True,mseconds=True):
        ''' Return a parsing of the time
        '''
        try :
            time.hours
            return (time.days,time.hours,time.minutes,time.seconds,time.mseconds)
        except:
            pass
        if split:
            time=time.split(' ')[0]
        if time.count(',')==0 and not check_only and mseconds:
            raise()
        time=re.split("[:.,]+",time)
        if not mseconds:
            time=time+[0]
        while len(time)<4:
            time=[0]+time
        hours,minutes,seconds,mseconds=[float(x) for x in time]
        hours+=int(minutes/60)
        minutes=minutes%60
        days=int(hours/24)
        hours=hours%24
        if not check_only:
            return (days,hours,minutes,seconds,mseconds)
        
    def get_seconds(self):  
        return self.days*3600*24+self.hours*3600 + self.minutes*60 + self.seconds + self.mseconds/1000
    
    def __str__(self):
        return str(int(self.days))+';'+str(int(self.hours))+':'+str(int(self.minutes))+'.'+str(int(self.seconds))+','+str(int(self.mseconds))
    
    def __div__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()-seconds_other
    
    def __truediv__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()/seconds_other
    

In [ ]:
dict_problems={}

In [ ]:
def apply_time(x,mseconds=True):
    try:
        time=Time(x,mseconds=mseconds)
    except:
        try:
            dict_problems[x]+=1
        except:
            dict_problems[x]=1
        return False
    return time

In [ ]:
df.temps=df.temps.apply(apply_time)

In [ ]:
df.retard=df.retard.apply(apply_time)

In [ ]:
df.pace=df.pace.apply(apply_time,args=(False,))

In [ ]:
print(Time('95.12,2'))

In [ ]:
a=Time('48.12,2')
b=Time('4.30,0')
print(a,b)

In [ ]:
b.get_seconds()

In [ ]:
a.get_seconds()

In [ ]:
a/b

In [ ]:
df['Distance']=df.temps/df.pace

In [ ]:
df.head(2)

In [ ]:
df_old=df.copy()

In [ ]:
df=df[df.Distance>0]

In [ ]:
df_old.shape

In [ ]:
df=df_old[df_old.Distance>0]

In [ ]:
df.head(3)

# Parse Date

In [ ]:
df['Year']=df.Date.apply(lambda x: x.split('.')[3])

In [ ]:
df['Month']=df.Date.apply(lambda x: x.split('.')[2])

In [ ]:
df.head(2)

# Sex Male-Female

### Get names database

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
gender_names=pd.read_csv('gender_names.csv')

In [ ]:
del gender_names['Unnamed: 0']

In [ ]:
gender_names=gender_names.set_index(gender_names.name)

In [ ]:
del gender_names['name']

In [ ]:
gender_names.head(2)

### Category from sex database

In [ ]:
category_list=list(df.cat.value_counts().index)

In [ ]:
len(category_list)

In [ ]:
category_dict_sex={}

In [ ]:
cat_to_check=['B','D','F40','F/3','H','12','10-F20']

In [ ]:
for i,cat in enumerate(category_list):
    if cat in category_dict_sex:
        continue
    names=list(df[df.cat==cat].nom.values)
    not_found=0
    male=0
    female=0
    for index,name in enumerate(names):
        splitted=re.split("[- ]+",name)
        found=0
        for name in splitted:
            try:
                row=gender_names.loc[name.lower()]
                male+=row.male_prob
                female+=row.female_prob
                found+=1
            except:
                pass
    if male!=0 or female!=0:
        male_perc=male/(male+female)
        category_dict_sex[cat]=male_perc
        if male_perc<0.85 and male_perc>0.15:
            if cat not in cat_to_check:
                cat_to_check.append(cat)
#             print('Category',cat,'Perc:',male_perc)
    if i%100==0:
#         print()
        print(i)
#         print()
    if i>7000:
        break

In [ ]:
len(cat_to_check)

In [ ]:
len(category_dict_sex)

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if category_dict_sex[cat]>0.85:
        cat_parsed+=1
        category_dict_sex[cat]='M'
    elif category_dict_sex[cat]<0.15:
        cat_parsed+=1
        category_dict_sex[cat]='F'

In [ ]:
category_dict_sex_copy=copy.deepcopy(category_dict_sex)

In [ ]:
import json

with open('category_dict_sex.json', 'w') as f:
    json.dump(category_dict_sex, f)

In [ ]:
import json

with open('category_dict_sex.json') as f:
    category_dict_sex = json.load(f)

### Improvements

In [ ]:
category_dict_sex=copy.deepcopy(category_dict_sex_copy)

In [ ]:
to_exclude=['P-W','D','SM2/D','W','P/W']

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if cat in to_exclude:
        continue
    if category_dict_sex[cat]=='M' or category_dict_sex[cat]=='F':
        cat_parsed+=1
        continue
    cat_splitted=re.split("[-/]+",cat)
    for i in cat_splitted:
        if len(i)==3 and (i[0]=='M' or i[0]=='H') and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        elif len(i)==3 and (i[0]=='F' or i[0]=='W')  and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Fille' in i or 'Women' in i or 'Dam' in i:
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Gar' in i or 'Men' in i or 'Hom' in i: 
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        if i=='W' or i=='D' or i=='H-U20W' or i=='H-WJ':
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
cat_parsed

In [ ]:
df[df.cat=='Sch-D-M']

### Final classification - accurate

In [ ]:
del df['sex']

In [ ]:
def add_sex(x):
    cat=x.cat.iloc[0]
    try:
        sex=category_dict_sex[cat]
        if sex=='M' or sex=='F':
            x['sex']=sex
        else:
            x['sex']=False
    except:
        x['sex']=False
    return x

In [ ]:
grouped=df.groupby(df.cat)

In [ ]:
df=grouped.apply(add_sex)

In [ ]:
df.sex.value_counts()

In [ ]:
df_sex=df[df.sex!=False]

In [ ]:
df=df_sex.copy()

In [ ]:
df_sex.head(2)

In [ ]:
df.shape

In [ ]:
df_sex.shape

In [ ]:
min(df.Year)

In [ ]:
min(df_sex.Year)

In [ ]:
df=df.rename(columns={'Year':'RaceYear','Month':'RaceMonth'})

In [ ]:
df.head(2)

In [ ]:
df.sex.value_counts()

# Parse year (roughly eliminating clearly bad entries)

In [ ]:
df.an=df.an.apply(lambda x: False if (x=='??' or x=='????' or x=='xxxx')  else x)

In [ ]:
df.an=df.an.apply(lambda x: int(x))

In [ ]:
df=df[df.an<2017]

In [ ]:
df=df[(df.an>1870) | (df.an<100)]

# Columns names

In [ ]:
df.head(1)

In [ ]:
df.columns.values

In [ ]:
df=df[['Name','Date','RaceYear','RaceMonth','cat','Distance','nom','sex','an','lieu','rang','temps','retard','pace']]

In [ ]:
df.head()

In [ ]:
df.columns=['Race','Date','RaceYear','RaceMonth','Category','Distance','Name','Sex','Year','LivingPlace','Rank','Time','Delay','Pace']

In [ ]:
df=df.reset_index()

In [ ]:
del df['index']

In [ ]:
df.head(2)

In [ ]:
df.shape

# Distance

In [ ]:
df.Distance=df.Distance.apply(lambda x: float(x))

In [ ]:
df['round']=df.Distance.round()

In [ ]:
df.Distance=df.Distance.apply(lambda x: x if (abs(x-round(x)))>0.065 else round(x))

In [ ]:
df.Distance=df.Distance.apply(lambda x: 21.0975 if (abs(x-21.0975))<0.1 else x)

In [ ]:
df.Distance=df.Distance.apply(lambda x: 42.195 if (abs(x-42.195))<0.1 else x)

In [ ]:
df.Distance.value_counts()

In [ ]:
del df['round']

In [ ]:
df.head(5)

# Adding weather informations

In [ ]:
weather_db=pd.read_csv('../datasets/races-information-weather.csv')
del weather_db['Unnamed: 0']
del weather_db['Unnamed: 0.1']

In [ ]:
weather_db.head(2)

In [ ]:
weather_db['RaceYear']=weather_db.Date.apply(lambda x: x.split('.')[3])

In [ ]:
gr=df.groupby([df.RaceYear,df.Race])

In [ ]:
def merge_weather_info(x):
    race=x.iloc[0].Race
    date=x.iloc[0].RaceYear
    weather=weather_db[(weather_db.Name==race) & (weather_db.RaceYear==date)].iloc[0]
    x['Place']=weather.Place
    x['MinTemp']=weather.min_temp
    x['MaxTemp']=weather.max_temp
    x['Weather']=weather.weather_desc
    x['RaceID']=weather.URL
    
    return x

In [ ]:
df=gr.apply(merge_weather_info)

In [ ]:
df.shape

In [ ]:
df.head(2)

# Parse times and select only meaningful features

In [ ]:
full_df=df.copy()

In [ ]:
base=datetime.datetime.strptime('0:0.0,0','%H:%M.%S,%f')

In [ ]:
full_df.Time=full_df.Time.apply(lambda x:str(x))

In [ ]:
full_df.Pace=full_df.Pace.apply(lambda x:str(x))

In [ ]:
def modify_to_parse_day(x):
    a=str(int(x[0])+1)
    return a+x[1:]

In [ ]:
full_df.Pace=full_df.Pace.apply(modify_to_parse_day)

In [ ]:
full_df.Time=full_df.Time.apply(modify_to_parse_day)

In [ ]:
full_df['Pace'] = (pd.to_datetime(full_df.Pace, format='%d;%H:%M.%S,%f')-base)

In [ ]:
max(full_df.Time)

In [ ]:
max(full_df.Distance)

In [ ]:
full_df['Time'] = (pd.to_datetime(full_df.Time, format='%d;%H:%M.%S,%f')-base)

In [ ]:
del full_df['Delay']

In [ ]:
full_df.head()

# Better distance processing

In [ ]:
df.Race.value_counts()

In [ ]:
full_df2=full_df.copy()

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: x if x<4 or (abs(1-x/round(x)))>0.0055 else round(x))

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 21.0975 if x>4 and (abs(1-x/21.0975))<0.0055 else x)

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 42.195 if x>4 and (abs(1-x/42.195))<0.0055 else x)

In [ ]:
full_df2.groupby(full_df2.RaceYear).count()

In [ ]:
df=full_df2.copy()

# Year - trying to get better data

In [ ]:
years_values=df.Year.value_counts().index.values
years_values[years_values<120]

In [ ]:
df.Year=df.Year.apply(lambda x: 0 if x==2 else x)

In [ ]:
df[df.Year>1900].shape

In [ ]:
df[df.Year<1900].shape

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
reduced_df[reduced_df.Year==25]

In [ ]:
reduced_df.groupby(reduced_df.Year).count()

If in the name a comma is present, it is possible that the year is misunderstood.

In [ ]:
names=reduced_df.Name
list_names=[]
for name in names:
    if ',' in name:
        list_names.append(name)

In [ ]:
df['CommaInName']=df.Name.str.contains(',')

In [ ]:
df['Less120']=df.Year<120

In [ ]:
df['FalseYear']=df.CommaInName & df.Less120

In [ ]:
def false_year(x):
    if x.iloc[0].FalseYear:
        x['Year']=float('nan')
    return x

In [ ]:
df=df.groupby(df.FalseYear).apply(false_year)

In [ ]:
del df['CommaInName']
del df['Less120']
del df['FalseYear']

In [ ]:
df.head(4)

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
list(reduced_df.Year.value_counts().index)

In [ ]:
df.Year=df.Year.apply(lambda x: float('nan') if x<10 else x)

In [ ]:
min(df.Year)

In [ ]:
df.Year=df.Year.apply(lambda x: x+1900 if x<120 else x)

# Output
First output

In [ ]:
df_output=df.copy()

In [ ]:
df_output.shape

In [ ]:
df_output.head()

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split(',')[0])

In [ ]:
df_output['len_name']=df_output.Name.apply(lambda x: len(x.rstrip().lstrip().split(' ')))

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split('+')[0])

In [ ]:
df_output

In [ ]:
df_output.len_name.value_counts()

In [ ]:
df_output[df_output.len_name==6]

In [ ]:
df_output.groupby(df_output.RaceYear).count()

In [ ]:
df.groupby(df.RaceYear).count()

In [ ]:
df_output.to_pickle('../datasets/intermediate_dataset.pickle')

# Solve problem 2 runners with same name in the same race

Strategy - study duplicates:  
- Remove all fully duplicated entries (same player in different categories)
- Keep entries with different living place (different players)
- To decide what to do if same place and different times

In [ ]:
df_output.Race.value_counts()

In [5]:
df_output=pd.read_pickle('../datasets/intermediate_dataset.pickle')

In [6]:
df_red=df_output.copy()#[df_output.Race=='Course de l\'Escalade, Genève']

In [ ]:
df_red.shape

In [ ]:
grouped=df_red.groupby([df_red.RaceID,df_red.Name,df_output.Year])

In [ ]:
duplicates=grouped.filter(lambda x: len(x)>1)

In [ ]:
duplicates.shape

In [ ]:
duplicates

In [ ]:
duplicates.Race.value_counts()

In [ ]:
a=df_output[(df_output.Race=='Jungfrau-Marathon, Interlaken') & (df_output.RaceYear==2012)]

In [ ]:
a[a.Name.to_series().str.contains('Baumer').tolist()]

In [ ]:
a[a.Name=='Peter Baumer']

In [ ]:
df_output[df_output.Name=='Baumer Peter']

In [ ]:
df_output.Race.value_counts()

In [ ]:
df_output[(df_output.Name=='Baumer Peter')]

In [ ]:
duplicates.columns

In [ ]:
d2=df_output[df_output.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'],keep=False)]

In [ ]:
d2=df_output[df_output.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace'],keep=False)]

In [ ]:
d2.shape

In [ ]:
d2[d2.Race=='Zürcher Silvesterlauf, Zürich']

In [ ]:
d2.Distance.value_counts()

In [ ]:
d1=duplicates[duplicates.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'],keep=False)]

In [ ]:
d1.shape

In [ ]:
d1.Race.value_counts()

In [ ]:
4961-4580

In [ ]:
duplicates.shape

In [ ]:
dup=duplicates.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'])

In [ ]:
dup.shape

In [ ]:
dup=duplicates.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth','Name', 'Sex',
       'Year', 'LivingPlace'],keep=False)

In [ ]:
dup.shape

In [ ]:
dup[dup.Race=='Jungfrau-Marathon, Interlaken']

In [7]:
df_output=df_output.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'])

# Sex verification
The sex classification seems working

In [ ]:
lau=df_output[(df_output.RaceID=='http://services.datasport.com/2007/lauf/Greifenseelauf')&(df_output.Distance>6)]

In [ ]:
lau.Distance.value_counts()

In [ ]:
lau.Sex.value_counts()

In [ ]:
df_output.RaceID.value_counts()

In [ ]:
df_output.shape

# Fixing not good categories

In [12]:
df_output.shape

(1650887, 19)

In [14]:
df=df_output.copy()

In [16]:
df.shape

(1650887, 19)

In [17]:
cat_list=list(df.Category.value_counts().index)

In [20]:
min(df.Pace)

Timedelta('0 days 00:01:20')

In [18]:
cat_list

['M40',
 'M20',
 'M50',
 'M30',
 'M45',
 'C',
 'H-M40',
 '13',
 '17-M20',
 'F40',
 'F20',
 'Hommes3',
 '10-M40',
 'Hommes2',
 '5-W20',
 'M60',
 'H-M30',
 'HS2',
 'A',
 '10-W20',
 'W40',
 'M-M40',
 '10-M20',
 '10-H30',
 'W20',
 'M55',
 '10-F20',
 'HS1',
 'Esc-Hom2',
 '10-W40',
 'H-M20',
 '10-F30',
 'M',
 '11',
 'Esc-Hom3',
 'W30',
 'H-M50',
 '10-M30',
 '10-H20',
 'M-M20',
 'F30',
 'W35',
 'HE',
 '5-W40',
 '10-M50',
 'Hommes4',
 'H/M40',
 'F50',
 'U',
 '21-M40',
 '10-W30',
 'H/M35',
 '5-W45',
 'Hommes1',
 'X',
 'W45',
 'Esc-Hom1',
 '20-H30',
 'M35',
 'HEcolB',
 '5-W35',
 'HPousA',
 'M-M50',
 'V',
 '10-W35',
 'HEcolA',
 'N',
 '10-W45',
 '5-W30',
 '21-M30',
 '10-H40',
 '17-M40',
 'F45',
 '10-F40',
 'H/M45',
 'MenAK1-8',
 '17-M45',
 'MenAK2-8',
 'H-F40',
 '8-MenAK2',
 '10-W50',
 'H/M30',
 'H-F30',
 '15-M20',
 '15-M45',
 'Esc-Hom4',
 '10-Frau',
 '15-M40',
 '21-H2',
 'C-M20',
 'H-Sen1',
 '8-MenAK1',
 '20-H20',
 '5-W50',
 'M-M30',
 'H-Sen2',
 'H-H3',
 '21-M50',
 'W50',
 '17-W20',
 'H-H2',
 'H-

In [26]:
df[df.Category=='MTBM1'].head(2)

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name


In [32]:
cat_to_eliminate=[]
for i in cat_list:
    if 'M-Grand' in i:
        print(i)
        cat_to_eliminate.append(i)

In [25]:
for i in cat_to_eliminate:
    df=df[df.Category!=i]

In [27]:
df.shape

(1650657, 19)

In [34]:
df_output=df.copy()

# Capitalization

In [35]:
'CIAO COME STAI'.title()

'Ciao Come Stai'

In [36]:
df=df_output.copy()

In [37]:
df.dtypes

Race                    object
Date                    object
RaceYear                object
RaceMonth               object
Category                object
Distance               float64
Name                    object
Sex                     object
Year                   float64
LivingPlace             object
Rank                     int64
Time           timedelta64[ns]
Pace           timedelta64[ns]
Place                   object
MinTemp                float64
MaxTemp                float64
Weather                 object
RaceID                  object
len_name                 int64
dtype: object

In [38]:
df.RaceYear=df.RaceYear.apply(lambda x: int(x))

In [39]:
df.RaceMonth=df.RaceMonth.apply(lambda x: int(x))

In [40]:
df.Name=df.Name.apply(lambda x: x.title())

In [41]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

In [42]:
df=df.reset_index()
del df['index']

In [44]:
df

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name
0,Kerzerslauf,sam. 18.03.2000,2000,3,M20,15.000000,Abgottspon Peter,M,1974.0,Zermatt,233,01:02:25,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
1,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Abplanalp Michael,M,1964.0,Bern,32,00:55:11.700000,00:03:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
2,Kerzerslauf,sam. 18.03.2000,2000,3,M50,15.000000,Abt Werner,M,1947.0,Spiez,155,01:12:42.900000,00:04:50,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
3,Kerzerslauf,sam. 18.03.2000,2000,3,F45,15.000000,Ackermann Antoinette,F,1953.0,Alterswil,48,01:22:36.700000,00:05:30,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
4,Kerzerslauf,sam. 18.03.2000,2000,3,F50,15.000000,Ackermann Hedy,F,1946.0,Alterswil FR,42,01:23:29.300000,00:05:33,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
5,Kerzerslauf,sam. 18.03.2000,2000,3,M40,15.000000,Ackermann Meinrad,M,1956.0,La Joux,35,00:56:53.100000,00:03:47,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
6,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Adamina Bruno,M,1962.0,Bern,26,00:54:41.700000,00:03:38,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
7,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Adatte Michel,M,1964.0,Asuel,48,00:56:36.600000,00:03:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
8,Kerzerslauf,sam. 18.03.2000,2000,3,M20,15.000000,Adesris Yeshaneh,M,1973.0,ETH-Aethiopien,23,00:49:27.300000,00:03:17,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2
9,Kerzerslauf,sam. 18.03.2000,2000,3,M50,15.000000,Adler Hanspeter,M,1949.0,Koblenz,33,01:02:23,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,2


# Output to file

In [49]:
del df_output['len_name']

In [50]:
df_output.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Name',
       'Sex', 'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID'],
      dtype='object')

In [51]:
df_output

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
0,Kerzerslauf,sam. 18.03.2000,2000,3,M20,15.000000,Abgottspon Peter,M,1974.0,Zermatt,233,01:02:25,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
1,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Abplanalp Michael,M,1964.0,Bern,32,00:55:11.700000,00:03:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
2,Kerzerslauf,sam. 18.03.2000,2000,3,M50,15.000000,Abt Werner,M,1947.0,Spiez,155,01:12:42.900000,00:04:50,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
3,Kerzerslauf,sam. 18.03.2000,2000,3,F45,15.000000,Ackermann Antoinette,F,1953.0,Alterswil,48,01:22:36.700000,00:05:30,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
4,Kerzerslauf,sam. 18.03.2000,2000,3,F50,15.000000,Ackermann Hedy,F,1946.0,Alterswil FR,42,01:23:29.300000,00:05:33,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
5,Kerzerslauf,sam. 18.03.2000,2000,3,M40,15.000000,Ackermann Meinrad,M,1956.0,La Joux,35,00:56:53.100000,00:03:47,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
6,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Adamina Bruno,M,1962.0,Bern,26,00:54:41.700000,00:03:38,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
7,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Adatte Michel,M,1964.0,Asuel,48,00:56:36.600000,00:03:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
8,Kerzerslauf,sam. 18.03.2000,2000,3,M20,15.000000,Adesris Yeshaneh,M,1973.0,ETH-Aethiopien,23,00:49:27.300000,00:03:17,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
9,Kerzerslauf,sam. 18.03.2000,2000,3,M50,15.000000,Adler Hanspeter,M,1949.0,Koblenz,33,01:02:23,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers


In [52]:
df_output.shape

(1650657, 18)

In [55]:
df_output[df_output.Year.isnull()]

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
1083,Kerzerslauf,sam. 18.03.2000,2000,3,F15,15.000000,Glanzmann Rosmarie,F,NaN,1943,3,01:29:06,00:05:56,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
3739,20km de Lausanne,sam. 29.04.2000,2000,4,13,20.000000,Oppliger Etienne,M,NaN,24 heures,22,01:41:30.700000,00:05:04,Lausanne,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/km20
3757,20km de Lausanne,sam. 29.04.2000,2000,4,13,20.000000,Visinand Andre,M,NaN,Team BCV 2 - 20 km,47,01:57:13.400000,00:05:51,Lausanne,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/km20
3758,20km de Lausanne,sam. 29.04.2000,2000,4,11,20.000000,Waldvogel Cathy,M,NaN,Iril SA,9,01:40:27.500000,00:05:01,Lausanne,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/km20
4146,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Barus Benson,M,NaN,Kenya,11,00:30:17.900000,00:03:01,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio
4147,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Belluschi Elio,M,NaN,Italia,9,00:30:05.600000,00:03:00,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio
4148,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Bernardini Severino,M,NaN,Italia,14,00:30:50.900000,00:03:05,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio
4149,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Bonetti Maurizio,M,NaN,Italia,18,00:31:42.700000,00:03:10,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio
4150,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Boudifa Mohamed,M,NaN,Algeria,19,00:31:44.600000,00:03:10,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio
4151,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Chicco Davide,M,NaN,I-Italia,16,00:31:14.700000,00:03:07,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio


In [54]:
df_output.dtypes

Race                    object
Date                    object
RaceYear                 int64
RaceMonth                int64
Category                object
Distance               float64
Name                    object
Sex                     object
Year                   float64
LivingPlace             object
Rank                     int64
Time           timedelta64[ns]
Pace           timedelta64[ns]
Place                   object
MinTemp                float64
MaxTemp                float64
Weather                 object
RaceID                  object
dtype: object

In [53]:
df_output.to_pickle('../datasets/df_added_category.pickle')